In [18]:
import os
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [19]:
# results_path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/gmark_50_new/mix"
results_path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/SPIN-to-N3/property-paths/zika"

## Load results

In [20]:
def load_sparql_times(system, data_size):
    sparql_times = f"{results_path}/results/{system}/{data_size}/times_{system}.csv"
    df_sparql = pd.read_csv(sparql_times)
    df_sparql['total_time'] = df_sparql['load_time'] + df_sparql['exec_time']
    
    return df_sparql


def load_sparql_times_agg(system, data_size):
    df_sparql = load_sparql_times(system, data_size)
    df_sparql_agg = df_sparql.groupby('query')[['load_time', 'exec_time', 'total_time']].mean().reset_index()
    df_sparql_agg['nr'] = df_sparql_agg['query'].str.slice(len("query-"), -len(".sparql"))
    
    return df_sparql_agg

In [21]:
def load_n3_times(type, data_size):
    n3_times = f"{results_path}/n3/results/{data_size}/times_n3.csv"
    df_n3 = pd.read_csv(n3_times)
    
    df_n3 = df_n3[df_n3['type']==type]
    
    df_n3['id'] = df_n3.apply(lambda x: f"{x['query']},{x['type']}", axis=1)
    df_n3_failed = df_n3[df_n3['reas_time']==-1]['id']
    df_n3_filt = df_n3[~df_n3['id'].isin(df_n3_failed)]
    
    df_n3_total = df_n3_filt[df_n3_filt['phase'].isna() | (df_n3_filt['phase']=='total')]
    df_n3_total['total_time'] = df_n3_total['netw_time'] + df_n3_total['reas_time']
    
    return df_n3_total


def load_n3_times_agg(type, data_size):
    df_n3_total = load_n3_times(type, data_size)    
    df_n3_agg = df_n3_total.groupby('query')[['netw_time', 'reas_time', 'total_time']].mean().reset_index()
    df_n3_agg['nr'] = df_n3_agg['query'].str.slice(len("query-"), -len(".n3"))

    return df_n3_agg

## Check outliers

In [12]:
def check_query_stdev(df, time_col, limit=10):
    for query in df['query'].unique():
        df_query = df[df['query']==query]
        stdev = df_query[time_col].describe()['std']
        if stdev > limit:
            print(query, "stdev:", stdev)
            print(df_query)
            print()

In [10]:
# check_query_stdev(load_sparql_times('jena', 50), time_col='exec_time', limit=50)
# check_query_stdev(load_sparql_times('jena', 100),time_col='exec_time',  limit=50)

## - jena, data_size 100:
# outliers (exec time): query-11 (100637 vs. avg. 153), query-2 (309138 vs. avg. 39551)
# (replaced with averages)

check_query_stdev(load_sparql_times('jena', "1_000_pt2"), time_col='exec_time', limit=50)

In [ ]:
check_query_stdev(load_sparql_times('rdflib', 50), time_col='exec_time', limit=50)

In [ ]:
check_query_stdev(load_sparql_times('rdf4j', 50), time_col='exec_time', limit=100)
check_query_stdev(load_sparql_times('rdf4j', 100), time_col='exec_time', limit=100)

In [13]:
# check_query_stdev(load_n3_times('fwd', 50), time_col='reas_time', limit=50)
# check_query_stdev(load_n3_times('fwd', 100), time_col='reas_time', limit=50)

check_query_stdev(load_n3_times('direct', "1_000_pt2"), time_col='reas_time', limit=50)
check_query_stdev(load_n3_times('bwd', "1_000_pt2"), time_col='reas_time', limit=50)
check_query_stdev(load_n3_times('fwd', "1_000_pt2"), time_col='reas_time', limit=50)

pp4.n3 stdev: 51.37087371912869
     run   query               data    type phase  netw_time  reas_time  \
35     0  pp4.n3  data_1_000_pt2.nt  direct   NaN        321       4459   
77     0  pp4.n3  data_1_000_pt2.nt  direct   NaN        304       4393   
119    1  pp4.n3  data_1_000_pt2.nt  direct   NaN        297       4392   
161    2  pp4.n3  data_1_000_pt2.nt  direct   NaN        312       4512   
203    3  pp4.n3  data_1_000_pt2.nt  direct   NaN        290       4392   
245    4  pp4.n3  data_1_000_pt2.nt  direct   NaN        298       4389   

                id  total_time  
35   pp4.n3,direct        4780  
77   pp4.n3,direct        4697  
119  pp4.n3,direct        4689  
161  pp4.n3,direct        4824  
203  pp4.n3,direct        4682  
245  pp4.n3,direct        4687  

pp3.n3 stdev: 62.42115026175023
     run   query               data type  phase  netw_time  reas_time  \
27     0  pp3.n3  data_1_000_pt2.nt  fwd  total        324       4997   
69     0  pp3.n3  data_1_000_pt2

## Check statistics

### Per query

In [28]:
load_sparql_times_agg('jena', '10_000_pt2').to_csv(f"{results_path}/../results/zika/per_query/jena-10_000.pt2.csv")

In [29]:
load_n3_times_agg('direct', '10_000_pt2').to_csv(f"{results_path}/../results/zika/per_query/n3_direct-10_000.pt2.csv")

In [30]:
load_n3_times_agg('fwd', '10_000_pt2').to_csv(f"{results_path}/../results/zika/per_query/n3_fwd-10_000.pt2.csv")

In [31]:
load_n3_times_agg('bwd', '10_000_pt2').to_csv(f"{results_path}/../results/zika/per_query/n3_bwd-10_000.pt2.csv")

### Overall

In [32]:
# desc_path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/SPIN-to-N3/property-paths/results/gmark"
desc_path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/SPIN-to-N3/property-paths/results/zika"

def describe_sparql(system, data_size, to_file=True):
    df = load_sparql_times_agg(system, data_size).describe()
    if not to_file:
        print(system, data_size, "\n", df)
    else:
        df.to_csv(os.path.join(desc_path, f"{system}-{data_size}.csv"))
    
def describe_n3(type, data_size, to_file=True):
    df = load_n3_times_agg(type, data_size).describe()
    if not to_file:
        print("n3", type, data_size, "\n", df)
    else:
        df.to_csv(os.path.join(desc_path, f"n3_{type}-{data_size}.csv"))

In [33]:
describe_sparql('jena', "1_000_pt2")
describe_sparql('jena', "10_000_pt2")
# describe_sparql('jena', "100_000_pt2")

In [35]:
describe_n3('direct', "10_000_pt2")
describe_n3('bwd', "10_000_pt2")
describe_n3('fwd', "10_000_pt2")

In [ ]:
describe_sparql('rdflib', 50)
describe_sparql('jena', 50)
describe_sparql('jena', 100)
describe_sparql('rdf4j', 50)
describe_sparql('rdf4j', 100)

In [ ]:
describe_n3('direct', 50)
describe_n3('bwd', 50)
describe_n3('fwd', 50)
describe_n3('fwd', 100)

In [ ]:
import matplotlib.pyplot as plt

def plot_systems(data_size, lim=None, incl_only=['n3_dir', 'n3_bwd', 'n3_fwd','jena', 'rdflib', 'rdf4j']):
    fig, ax = plt.subplots()
    if lim is not None:
        ax.set_ylim(0, lim)

    if 'n3_fwd' in incl_only:
        load_n3_times_agg('fwd', data_size).plot(x='nr', y='reas_time', ax=ax, label='fwd')
    if 'n3_bwd' in incl_only:
        load_n3_times_agg('bwd', data_size).plot(x='nr', y='reas_time', ax=ax, label='bwd')
    if 'n3_dir' in incl_only:
        load_n3_times_agg('direct', data_size).plot(x='nr', y='reas_time', ax=ax, label='direct')

    if 'jena' in incl_only:
        load_sparql_times_agg('jena', data_size).plot(x='nr', y='exec_time', ax=ax, label='jena')
    if 'rdflib' in incl_only:
        load_sparql_times_agg('rdflib', data_size).plot(x='nr', y='exec_time', ax=ax, label='rdflib')
    if 'rdf4j' in incl_only:
        load_sparql_times_agg('rdf4j', data_size).plot(x='nr', y='exec_time', ax=ax, label='rdf4j')

In [ ]:
plot_systems(50, lim=4000)

In [ ]:
plot_systems(50, lim=500, incl_only=['n3_fwd', 'jena', 'rdf4j'])

In [ ]:
plot_systems(100, lim=1000, incl_only=['n3_fwd', 'jena', 'rdf4j'])